In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm # progress bar
import copy

#### Make the sum of all the datasets


In [ ]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/TR/data_tr'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/TR/data_tr/INCART 2-lead Arrhythmia Database.csv
/content/drive/MyDrive/TR/data_tr/MIT-BIH Arrhythmia Database.csv
/content/drive/MyDrive/TR/data_tr/MIT-BIH Supraventricular Arrhythmia Database.csv
/content/drive/MyDrive/TR/data_tr/Sudden Cardiac Death Holter Database.csv


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TR/data_tr/MIT-BIH Arrhythmia Database.csv')
df2 = pd.read_csv('/content/drive/MyDrive/TR/data_tr/INCART 2-lead Arrhythmia Database.csv')
df3 = pd.read_csv('/content/drive/MyDrive/TR/data_tr/MIT-BIH Supraventricular Arrhythmia Database.csv')
df4 = pd.read_csv('/content/drive/MyDrive/TR/data_tr/Sudden Cardiac Death Holter Database.csv')

print(df.shape)
df.head()

(100689, 34)


<ipython-input-43-6ec1af9b1b4d>:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv('/content/drive/MyDrive/TR/data_tr/Sudden Cardiac Death Holter Database.csv')


,record,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,101,N,76,313.0,0.074347,-0.160548,1.036401,-0.285662,-0.026824,41,...,0.025930,2,18,22,2,0.025930,0.025930,0.025930,0.025436,0.025436
1,101,N,313,315.0,-0.052079,-0.264784,0.886597,-0.366298,-0.059710,21,...,-0.042009,26,27,62,9,-0.042009,-0.029498,0.005012,0.030892,0.002986
2,101,N,315,321.0,-0.062151,-0.296983,0.991859,-0.410306,-0.065686,22,...,0.009528,3,8,12,1,0.009528,0.009528,0.008786,0.008786,0.008368
3,101,N,321,336.0,-0.063322,-0.281386,1.034903,-0.403880,-0.071750,22,...,-0.020536,6,9,16,1,-0.020536,-0.020257,-0.018965,-0.016968,-0.014555
4,101,N,336,344.0,-0.062915,1.046914,1.046408,1.046408,-0.074639,11,...,0.016053,16,5,31,10,0.016053,0.006742,0.002782,-0.007798,-0.051155


#### Carga los datasets en general

del 2 3 4 solo se cogen los casos positivos y del primero de los normales solo hasta igualar los casos con problemas
sustiuye los valores y y crea las las labels

In [ ]:
df = df[df.notnull().all(axis=1)]
df2 = df2[df2.notnull().all(axis=1)]
df3 = df3[df3.notnull().all(axis=1)]
df4 = df4[df4.notnull().all(axis=1)]


y_label = df[['type']]
y_label.replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
y_label.replace(['N'], 'normal', inplace=True)

y_label2 = df2[['type']]
y_label2.replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
y_label2.replace(['N'], 'normal', inplace=True)

y_label3 = df3[['type']]
y_label3.replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
y_label3.replace(['N'], 'normal', inplace=True)

y_label4 = df4[['type']]
y_label4.replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
y_label4.replace(['N'], 'normal', inplace=True)

y_label2.value_counts()


<ipython-input-44-2b964ae7b4fe>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_label.replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
<ipython-input-44-2b964ae7b4fe>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_label.replace(['N'], 'normal', inplace=True)
<ipython-input-44-2b964ae7b4fe>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_label2.replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
<ipyt

,count
type,
normal,153546
arrhythmia,22183


In [ ]:
df[['type']].replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
df[['type']].replace(['N'], 'normal', inplace=True)
df['type'].value_counts()

<ipython-input-45-4f0c0ae7d645>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['type']].replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia', inplace=True)
<ipython-input-45-4f0c0ae7d645>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['type']].replace(['N'], 'normal', inplace=True)


,count
type,
N,90083
VEB,7009
SVEB,2779
F,803
Q,15


In [ ]:
num_arrythmia = len(y_label[y_label['type']=='arrhythmia']) + len(y_label2[y_label2['type']=='arrhythmia']) + len(y_label3[y_label3['type']=='arrhythmia']) + len(y_label4[y_label4['type']=='arrhythmia'])
num_arrythmia

68111

In [ ]:
y_label.value_counts()

,count
type,
normal,90083
arrhythmia,10606


In [ ]:
arr1 = df.loc[y_label['type']=='arrhythmia']
nor1 = df.loc[y_label['type']=='normal'][:num_arrythmia//4]

arr2 = df2.loc[y_label2['type']=='arrhythmia']
nor2 = df2.loc[y_label2['type']=='normal'][:num_arrythmia//4]

arr3 = df3.loc[y_label3['type']=='arrhythmia']
nor3 = df3.loc[y_label3['type']=='normal'][:num_arrythmia//4]

arr4 = df4.loc[y_label4['type']=='arrhythmia']
nor4 = df4.loc[y_label4['type']=='normal'][:num_arrythmia//4]


In [ ]:
x_data = pd.concat([arr1.iloc[:, 2:], nor1.iloc[:, 2:], arr2.iloc[:, 2:], nor2.iloc[:, 2:], arr3.iloc[:, 2:], nor3.iloc[:, 2:], arr4.iloc[:, 2:], nor4.iloc[:, 2:]])
y_label = pd.concat( [nor1[['type']], nor2[['type']], nor3[['type']], nor4[['type']], y_label.loc[y_label['type'] == 'arrhythmia'], y_label2.loc[y_label2['type'] == 'arrhythmia'], y_label3.loc[y_label3['type'] == 'arrhythmia'], y_label4.loc[y_label4['type'] == 'arrhythmia']])


In [ ]:
y_label.value_counts()

,count
type,
N,68108
arrhythmia,57505


In [ ]:
df1normal = df.loc[y_label['type']=='normal'][:125616]
df1normal

y_label_normal = df1normal[['type']]
x_data_normal = df1normal.iloc[:, 2:]



### Make the split into the features and the labels

In [ ]:
x_data = df.iloc[:, 2:]

In [ ]:
y_label.value_counts()

type      
normal        90083
arrhythmia    10606
Name: count, dtype: int64

In [ ]:
print(len(x_data))
print(len(y_label))

100689
100689


In [ ]:
a = x_data.loc[y_label['type'] == 'arrhythmia']

In [ ]:
y_label = pd.concat([y_label_normal, y_label.loc[y_label['type'] == 'arrhythmia'],y_label2.loc[y_label2['type'] == 'arrhythmia'], y_label3.loc[y_label3['type'] == 'arrhythmia'], y_label4.loc[y_label4['type'] == 'arrhythmia']])
x_data = pd.concat([x_data_normal, a, df2.loc[y_label2['type'] == 'arrhythmia'].iloc[:, 2:], df3.loc[y_label3['type'] == 'arrhythmia'].iloc[:, 2:] , df4.loc[y_label4['type'] == 'arrhythmia'].iloc[:, 2:] ])

In [ ]:
y_label.value_counts()

type      
N             90083
arrhythmia    68111
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.2, random_state=33)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)

y_train_scaled = encoder.transform(y_train)
y_test_scaled = encoder.transform(y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


#### Transform to tensors

In [ ]:
X_train = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train_scaled, dtype=torch.float32).reshape(-1, 1)

X_test = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test_scaled, dtype=torch.float32).reshape(-1, 1)

In [ ]:
len(x_train_scaled[0])

32

### Define the binary classificator

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(32, 24)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(24, 24)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(24, 10)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [ ]:
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 25   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)
    batches_per_epoch = len(X_train) // batch_size

    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
      model.train()
      for i in range(batches_per_epoch):
        start = i * batch_size
        Xbatch = X_train[start:start+batch_size]
        ybatch = y_train[start:start+batch_size]
        # forward pass
        y_pred = model(Xbatch)
        loss = loss_fn(y_pred, ybatch)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()


      # evaluate trained model with test set
      model.eval()
      with torch.no_grad():
        y_pred = model(X_val)
      accuracy = (y_pred.round() == y_val).float().mean()
      print("Accuracy {:.2f}".format(accuracy * 100))


    # restore model and return best accuracy
    #model.load_state_dict(best_weights)
    #return best_acc

In [ ]:
model = Model()

model_train(model, X_train, y_train, X_test, y_test)

NameError: name 'model_train' is not defined

In [ ]:
model = Model()

model_train(model, X_train, y_train, X_test, y_test)

Accuracy 92.63
Accuracy 93.49
Accuracy 93.73
Accuracy 93.69
Accuracy 93.66
Accuracy 93.76
Accuracy 93.80
Accuracy 93.79
Accuracy 93.80
Accuracy 93.82
Accuracy 93.86
Accuracy 93.87
Accuracy 93.88
Accuracy 93.93
Accuracy 94.10
Accuracy 94.21
Accuracy 94.22
Accuracy 94.33
Accuracy 94.32
Accuracy 94.38
Accuracy 94.44


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/TR/heartbeatvision2.pt')

In [ ]:
with torch.no_grad():
    y_val = model(X_test)


correct = 0

for i in range(len(X_test)):
    print(f' Predicted: ', round(y_val[i].item()),  ' Real: ', y_test[i].item()  )


    if round(y_val[i].item()) == y_test[i].item():
        correct += 1
    #    print(correct)

accuracy = correct / len(X_test) * 100
print(f'The accuracy is {accuracy} %')




Se han truncado las últimas 5000 líneas del flujo de salida.
 Predicted:  0  Real:  0.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  0  Real:  0.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  0  Real:  0.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  0  Real:  0.0
 Predicted:  1  Real:  1.0
 Predicted:  1  Real:  1.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  1  Real:  1.0
 Predicted:  0  Real:  0.0
 Predicted:  1  Real:  1.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:  0.0
 Predicted:  0  Real:

In [ ]:
model = Model()

model.load_state_dict(torch.load('/content/drive/MyDrive/TR/heartbeatvision2.pt'))